In [36]:
root_path = "C:\\Users\\user\\Desktop\\DataBackup\\S09\\"


# filtering bandwidth
low, high = 0.1, 30

resampleRate = 100

In [37]:
import numpy as np
import pickle
import pandas as pd
import statsmodels.api as sm
import hdf5storage
from scipy.signal import butter, lfilter
import os, glob, time
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from datetime import datetime
from sklearn.externals import joblib
from scipy import signal
import matplotlib.pyplot as plt
import random

def Re_referencing(eegData, channelNum, sampleNum):
        after_car = np.zeros((channelNum,sampleNum))
        for i in np.arange(channelNum):
            after_car[i,:] = eegData[i,:] - np.mean(eegData,axis=0)
        return after_car
    
def butter_bandpass(lowcut, highcut, fs, order=5):
        nyq = 0.5 * fs
        low = lowcut / nyq
        high = highcut / nyq
        b, a = butter(order, [low, high], btype='band')
        return b, a
def butter_bandpass_filter(data, lowcut, highcut, fs, order=5):
        b, a = butter_bandpass(lowcut, highcut, fs, order=order)
        y = lfilter(b, a, data)
        return y

def Epoching(eegData, stims, code, samplingRate, nChannel, epochSampleNum, epochOffset,baseline):
        Time = stims[np.where(stims[:,1] == code),0][0]
        Time = np.floor(np.multiply(Time,samplingRate)).astype(int)
        Time_after = np.add(Time,epochOffset).astype(int)
        Time_base = np.add(Time,baseline).astype(int)
        Num = Time.shape
        Epochs = np.zeros((Num[0], nChannel, epochSampleNum))
        for j in range(Num[0]):
            Epochs[j, :, :] = eegData[:, Time_after[j]:Time_after[j] + epochSampleNum]
            
            for i in range(nChannel):
                Epochs[j,i,:] = Epochs[j,i,:] - np.mean(Epochs[j,i,:])
                
        return [Epochs, Num[0]]

def Convert_to_featureVector(EpochsT, NumT, EpochsN, NumN, featureNum):
        FeaturesT = np.zeros((NumT, featureNum))
        for i in range(NumT):
            FeaturesT[i,:] = np.reshape(EpochsT[i,:,:],(1,featureNum))
        FeaturesN = np.zeros((NumN, featureNum))
        for j in range(NumN):
            FeaturesN[j,:] = np.reshape(EpochsN[j,:,:],(1,featureNum))
        return [FeaturesT,FeaturesN]
    
def Standardization(Epochs):
    for i in range(Epochs.shape[1]):
        Epochs[:,i,:] = np.subtract(Epochs[:,i,:], np.mean(Epochs[:,i,:]))
        Epochs[:,i,:] = Epochs[:,i,:] / np.std(Epochs[:,i,:])
    
    return Epochs

def Make_Average_Component(EpochsT, NumT, EpochsN, NumN, channelNum, epochSampleNum, componentNum):
#     EpochsT = Standardization(EpochsT)
#     EpochsN = Standardization(EpochsN)
    
    NumT_Aver = NumT-componentNum
    NumN_Aver = NumN-componentNum
    
    EpochsT_Aver = np.zeros((NumT_Aver, channelNum, epochSampleNum))
    EpochsN_Aver = np.zeros((NumN_Aver, channelNum, epochSampleNum))
    for i in range(NumT_Aver):
        EpochsT_Aver[i, :, :] = np.mean(EpochsT[i:i+componentNum, :, :], axis=0)
    for j in range(NumN_Aver):
        EpochsN_Aver[j, :, :] = np.mean(EpochsN[j:j+componentNum, :, :], axis=0)
        
    return [EpochsT_Aver, NumT_Aver, EpochsN_Aver, NumN_Aver]

def resampling(Epochs, EpochNum, resampleRate, channelNum):
        resampled_epoch = np.zeros((EpochNum, channelNum, resampleRate))
        for i in range(EpochNum):
            for j in range(channelNum):
                resampled_epoch[i,j,:] = signal.resample(Epochs[i,j,:], resampleRate)
        return resampled_epoch
    
def main():
        start = time.time()
        
        ##Generate Preprocessing Training data
        ctime = datetime.today().strftime("%m%d_%H%M")
        Classifier_path = 'C:\\Users\\user\\Desktop\\Model\\' + ctime + 'Classifier.pickle'
        
        mat_path = root_path + 'Training\\mat\\'
        current_list = sorted(glob.glob(mat_path + '*.mat'), key=os.path.getmtime)
        
        matfile_name = current_list[0]
        
        mat = hdf5storage.loadmat(matfile_name)
        channelNames = mat['channelNames']
        eegData = mat['eegData']
        samplingFreq = mat['samplingFreq']
        samplingFreq = samplingFreq[0,0]
        stims = mat['stims']
        channelNum = channelNames.shape
        channelNum = channelNum[1]
        eegData = np.transpose(eegData)
        
        ##Preprocessing process
        
        #Bandpass Filter
        eegData = butter_bandpass_filter(eegData, low, high, samplingFreq, order=4)
    
        #Epoching
        epochSampleNum = int(np.floor(1.0 * samplingFreq))
        offset = int(np.floor(0.0 * samplingFreq)) 
        baseline = int(np.floor(1.0 * samplingFreq)) 
        [EpochsT, NumT] = Epoching(eegData, stims, 1, samplingFreq, channelNum, epochSampleNum, offset, baseline)
        [EpochsN, NumN] = Epoching(eegData, stims, 0, samplingFreq, channelNum, epochSampleNum, offset, baseline)
        
#         EpochsN_New = np.zeros((NumT, channelNum, epochSampleNum))
#         NumN = NumT
#         for i in range(NumN):
#             EpochsN_New[i,:,:] = np.mean(EpochsN[i*5:i*5+5, :, :], axis=0)
        
#         EpochsN = EpochsN_New

        #Convert to feature vector
        [EpochsT, NumT, EpochsN, NumN] = Make_Average_Component(EpochsT, NumT, EpochsN, NumN, channelNum, epochSampleNum, 20)
        EpochsT = resampling(EpochsT, NumT, resampleRate, channelNum) 
        EpochsN = resampling(EpochsN, NumN, resampleRate, channelNum)
        
        featureNum = channelNum*resampleRate
        
        [FeaturesT, FeaturesN] = Convert_to_featureVector(EpochsT, NumT, EpochsN, NumN, featureNum)
        TrainData = np.concatenate((FeaturesT, FeaturesN))
        TrainLabel = np.concatenate((np.ones((NumT,1)).astype(int),np.zeros((NumN,1)).astype(int))).ravel()

        #Saving LDA classifier
        lda = LinearDiscriminantAnalysis(solver='lsqr',shrinkage='auto')
        lda.fit(TrainData, TrainLabel)
        joblib.dump(lda, Classifier_path, protocol=2)
        
        print("time :", time.time() - start)
        
if __name__ == "__main__":
    main()

time : 0.6911518573760986


In [38]:
import numpy as np
from scipy.signal import butter, lfilter, sosfiltfilt
import time
import os, glob
import hdf5storage
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.externals import joblib
import shutil
from datetime import datetime
import socket
from scipy import signal
import matplotlib.pyplot as plt

def Re_referencing(eegData, channelNum, sampleNum):
        after_car = np.zeros((channelNum,sampleNum))
        for i in np.arange(channelNum):
            after_car[i,:] = eegData[i,:] - np.mean(eegData,axis=0)
        return after_car
    
def butter_bandpass(lowcut, highcut, fs, order=5):
        nyq = 0.5 * fs
        low = lowcut / nyq
        high = highcut / nyq
        b, a = butter(order, [low, high], btype='band')
        return b, a
def butter_bandpass_filter(data, lowcut, highcut, fs, order=5):
        b, a = butter_bandpass(lowcut, highcut, fs, order=order)
        y = lfilter(b, a, data)
        return y

def Standardization(Epochs):
    for i in range(Epochs.shape[1]):
        Epochs[:,i,:] = np.subtract(Epochs[:,i,:], np.mean(Epochs[:,i,:]))
        Epochs[:,i,:] = Epochs[:,i,:] / np.std(Epochs[:,i,:])
    
    return Epochs    
    
def Epoching(eegData, stims, code, samplingRate, nChannel, epochSampleNum, epochOffset,baseline):
        Time = stims[np.where(stims[:,1] == code),0][0]
        Time = np.floor(np.multiply(Time,samplingRate)).astype(int)
        Time_after = np.add(Time,epochOffset).astype(int)
        Time_base = np.add(Time,baseline).astype(int)
        Num = Time.shape
        Epochs = np.zeros((Num[0], nChannel, epochSampleNum))
        for j in range(Num[0]):
            Epochs[j, :, :] = eegData[:, Time_after[j]:Time_after[j] + epochSampleNum]
            
            for i in range(nChannel):
                Epochs[j,i,:] = Epochs[j,i,:] - np.mean(Epochs[j,i,:])
                
#         Epochs = Standardization(Epochs)
        Epochs_Aver = np.mean(Epochs, axis=0)

        return Epochs_Aver
    
def Convert_to_FeatureVector(Epochs, buttonNum, featureNum):
    Features = np.zeros((buttonNum, featureNum))
    for i in range(buttonNum):
        Features[i, :] = np.reshape(Epochs[i, :, :], (1, featureNum))
    return Features

def resampling(Epochs, resampleRate, channelNum):
        resampled_epoch = np.zeros((channelNum, resampleRate))
        for j in range(channelNum):
            resampled_epoch[j,:] = signal.resample(Epochs[j,:], resampleRate)
            
        return resampled_epoch
    
def main():
        Classifier_path = 'C:\\Users\\user\\Desktop\\Model\\'
        classifier_list = sorted(glob.glob(Classifier_path + '*.pickle'), key=os.path.getmtime, reverse=True)
        print("classifer:", classifier_list[0])

        lda = joblib.load(classifier_list[0])
        
        
#         classifier = "C:\\Users\\user\\Desktop\\DataBackup\\S02\\1114_1315Classifier.pickle"        
#         lda = joblib.load(classifier)


        mat_path = root_path + 'Online\\mat\\'
        current_list = sorted(glob.glob(mat_path + '*.mat'), key=os.path.getmtime)
        score = 0
        wrong_ans = []
        
        target_val = nontarget_val = 0
        
        for mat_file in current_list:
            print(mat_file)
            ans = mat_file[-5]
            
            mat = hdf5storage.loadmat(mat_file)
            channelNames = mat['channelNames']
            eegData = mat['eegData']
            samplingFreq = mat['samplingFreq']
            samplingFreq = samplingFreq[0,0]
            stims = mat['stims']
            channelNum = channelNames.shape
            channelNum = channelNum[1]
            eegData = np.transpose(eegData)
            buttonNum = 7
            
            #Bandpass Filter
            eegData = butter_bandpass_filter(eegData, low, high, samplingFreq, order=4)

            #Epoching
            epochSampleNum = int(np.floor(1.0 * samplingFreq))
            offset = int(np.floor(0.0 * samplingFreq))
            baseline = int(np.floor(1.0 * samplingFreq))
            
            ####### averaging whole epochs
            Epochs_Aver = np.zeros((buttonNum, channelNum, epochSampleNum))
            Epochs_final = np.zeros((buttonNum, channelNum, resampleRate))
            
            featureNum = channelNum*resampleRate
            
            for i in range(buttonNum):
                Epochs_Aver[i] = Epoching(eegData, stims, (i+1), samplingFreq, channelNum, epochSampleNum, offset, baseline)
                Epochs_final[i] = resampling(Epochs_Aver[i], resampleRate, channelNum)
            
            Features = Convert_to_FeatureVector(Epochs_final, buttonNum, featureNum)
        
            Answers = lda.decision_function(Features)
            w_norm = np.linalg.norm(lda.coef_)
            dist = Answers / w_norm
            
#             print(dist)
            
            for i in range(buttonNum):
                if i == int(ans)-1:
                    target_val = target_val + Answers[i]
                else:
                    nontarget_val = nontarget_val + Answers[i]
            
            answer = np.argmax(Answers) + 1
            
#             print(Answers)
            if int(ans) == int(answer): 
                score = score + 1
            else:
                wrong_ans.append(mat_file[mat_file.rfind('\\')+1:mat_file.rfind('_')])
            print('order: ', ans, 'predict: ', answer)
        
        print('wrong answer', wrong_ans)
        print('score:', score, '/', len(current_list))

        print('target mean:', target_val / len(current_list))
        print('nontarget mean:', nontarget_val / (len(current_list)*6))
    
if __name__ == "__main__":
    main()

classifer: C:\Users\user\Desktop\Model\1117_2319Classifier.pickle
C:\Users\user\Desktop\DataBackup\S09\Online\mat\1_6.mat
order:  6 predict:  6
C:\Users\user\Desktop\DataBackup\S09\Online\mat\2_4.mat
order:  4 predict:  4
C:\Users\user\Desktop\DataBackup\S09\Online\mat\3_3.mat
order:  3 predict:  5
C:\Users\user\Desktop\DataBackup\S09\Online\mat\4_7.mat
order:  7 predict:  7
C:\Users\user\Desktop\DataBackup\S09\Online\mat\5_1.mat
order:  1 predict:  1
C:\Users\user\Desktop\DataBackup\S09\Online\mat\6_5.mat
order:  5 predict:  5
C:\Users\user\Desktop\DataBackup\S09\Online\mat\7_2.mat
order:  2 predict:  7
C:\Users\user\Desktop\DataBackup\S09\Online\mat\8_1.mat
order:  1 predict:  1
C:\Users\user\Desktop\DataBackup\S09\Online\mat\9_3.mat
order:  3 predict:  3
C:\Users\user\Desktop\DataBackup\S09\Online\mat\10_7.mat
order:  7 predict:  7
C:\Users\user\Desktop\DataBackup\S09\Online\mat\11_7.mat
order:  7 predict:  7
C:\Users\user\Desktop\DataBackup\S09\Online\mat\12_7.mat
order:  7 predict

In [30]:
import numpy as np
from scipy.signal import butter, lfilter, sosfiltfilt
import time
import os, glob
import hdf5storage
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.externals import joblib
import shutil
from datetime import datetime
import socket
from scipy import signal
import matplotlib.pyplot as plt

def Re_referencing(eegData, channelNum, sampleNum):
        after_car = np.zeros((channelNum,sampleNum))
        for i in np.arange(channelNum):
            after_car[i,:] = eegData[i,:] - np.mean(eegData,axis=0)
        return after_car
    
def butter_bandpass(lowcut, highcut, fs, order=5):
        nyq = 0.5 * fs
        low = lowcut / nyq
        high = highcut / nyq
        b, a = butter(order, [low, high], btype='band')
        return b, a
def butter_bandpass_filter(data, lowcut, highcut, fs, order=5):
        b, a = butter_bandpass(lowcut, highcut, fs, order=order)
        y = lfilter(b, a, data)
        return y

def Standardization(Epochs):
    for i in range(Epochs.shape[1]):
        Epochs[:,i,:] = np.subtract(Epochs[:,i,:], np.mean(Epochs[:,i,:]))
        Epochs[:,i,:] = Epochs[:,i,:] / np.std(Epochs[:,i,:])
    
    return Epochs    
    
def Epoching(eegData, stims, code, samplingRate, nChannel, epochSampleNum, epochOffset,baseline):
        Time = stims[np.where(stims[:,1] == code),0][0]
        Time = np.floor(np.multiply(Time,samplingRate)).astype(int)
        Time_after = np.add(Time,epochOffset).astype(int)
        Time_base = np.add(Time,baseline).astype(int)
        Num = Time.shape
        Epochs = np.zeros((Num[0], nChannel, epochSampleNum))
        for j in range(Num[0]):
            Epochs[j, :, :] = eegData[:, Time_after[j]:Time_after[j] + epochSampleNum]
            
#             for i in range(nChannel):
#                 Epochs[j,i,:] = Epochs[j,i,:] - np.mean(Epochs[j,i,:])
                
        Epochs = Standardization(Epochs)
#         Epochs_Aver = np.mean(Epochs, axis=0)

        return Epochs
    
def Convert_to_FeatureVector(Epochs, buttonNum, featureNum):
    Features = np.zeros((buttonNum, 20, featureNum))
    for i in range(buttonNum):
        for j in range(20):
            Features[i, j, :] = np.reshape(Epochs[i, j, :, :], (1, featureNum))
    return Features

def resampling(Epochs, resampleRate, channelNum):
        resampled_epoch = np.zeros((20, channelNum, resampleRate))
        for i in range(20):
            for j in range(channelNum):
                resampled_epoch[i, j,:] = signal.resample(Epochs[i, j,:], resampleRate)
            
        return resampled_epoch
    
def main():
        Classifier_path = 'C:\\Users\\hyuns\\Desktop\\2020-2\\캡스톤\\LDAModel\\New\\'
        classifier_list = sorted(glob.glob(Classifier_path + '*.pickle'), key=os.path.getmtime, reverse=True)
        print("classifer:", classifier_list[0])
        
        lda = joblib.load(classifier_list[0])
        
        mat_path = root_path + 'Online\\'
        current_list = sorted(glob.glob(mat_path + '*.mat'), key=os.path.getmtime)
        score = 0
        wrong_ans = []
        
        for mat_file in current_list:
            print(mat_file)
            ans = mat_file[-5]
            
            mat = hdf5storage.loadmat(mat_file)
            channelNames = mat['channelNames']
            eegData = mat['eegData']
            samplingFreq = mat['samplingFreq']
            samplingFreq = samplingFreq[0,0]
            stims = mat['stims']
            channelNum = channelNames.shape
            channelNum = channelNum[1]
            eegData = np.transpose(eegData)
            buttonNum = 7
            
            sampleNum = eegData.shape[1]
        
            #Common Average Reference
#             eegData = Re_referencing(eegData, channelNum, sampleNum)
            
            #Bandpass Filter
            eegData = butter_bandpass_filter(eegData, low, high, samplingFreq, order=4)

            #Epoching
            epochSampleNum = int(np.floor(1.0 * samplingFreq))
            offset = int(np.floor(0.0 * samplingFreq))
            baseline = int(np.floor(1.0 * samplingFreq))
            
            ####### averaging whole epochs
            Epochs_Aver = np.zeros((buttonNum, 20, channelNum, epochSampleNum))
            Epochs_final = np.zeros((buttonNum, 20, channelNum, resampleRate))
            
            featureNum = channelNum*resampleRate
            
            for i in range(buttonNum):
                Epochs_Aver[i] = Epoching(eegData, stims, (i+1), samplingFreq, channelNum, epochSampleNum, offset, baseline)
                Epochs_final[i] = resampling(Epochs_Aver[i], resampleRate, channelNum)
            
            Features = Convert_to_FeatureVector(Epochs_final, buttonNum, featureNum)
            
            
            Ans_arr = np.zeros(7)
            for i in range(buttonNum):
                Ans_arr[i] = np.mean(lda.decision_function(Features[i]))
                
                
            print(Ans_arr)
            
            answer = np.argmax(Ans_arr) + 1
            
#             print(Answers)
            if int(ans) == int(answer): 
                score = score + 1
            else:
                wrong_ans.append(mat_file[mat_file.rfind('\\')+1:mat_file.rfind('_')])
            print('order: ', ans, 'predict: ', answer)
        
        print('wrong answer', wrong_ans)
        print('score:', score, '/', len(current_list))
            
if __name__ == "__main__":
    main()

classifer: C:\Users\hyuns\Desktop\2020-2\캡스톤\LDAModel\New\1117_2042Classifier.pickle
C:\Users\hyuns\Desktop\2020-2\캡스톤\EEGData\VR300\S07\Online\1_1.mat
[ 200.70341418 -222.78682004  -68.34929716 -241.48387254  -71.79164483
 -215.30511422 -271.44374945]
order:  1 predict:  1
C:\Users\hyuns\Desktop\2020-2\캡스톤\EEGData\VR300\S07\Online\2_2.mat
[-110.0754769    56.06359598 -321.03506551  -64.72824158 -207.43441375
  -71.08502178 -119.51991695]
order:  2 predict:  2
C:\Users\hyuns\Desktop\2020-2\캡스톤\EEGData\VR300\S07\Online\3_3.mat
[-134.84594165 -266.44347415  133.67301724 -102.44782322 -228.13380602
  -41.57150885  -48.46730189]
order:  3 predict:  3
C:\Users\hyuns\Desktop\2020-2\캡스톤\EEGData\VR300\S07\Online\4_4.mat
[-135.99748648 -129.45551689  -64.24965329 -140.14263325 -180.53440799
 -104.68705524 -225.0848856 ]
order:  4 predict:  3
C:\Users\hyuns\Desktop\2020-2\캡스톤\EEGData\VR300\S07\Online\5_5.mat
[-138.36462629 -203.02298571  -59.86691591 -128.57199845  -38.64474266
 -392.67720259 -2